In [1]:
import requests
import pandas as pd
import math
import aiohttp
import asyncio

from typing import List, Optional, Dict, TypeAlias
JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None

import nest_asyncio
nest_asyncio.apply()

In [20]:
def build_treasurydirect_header(
    host_str: str = "api.fiscaldata.treasury.gov", cookie_str: Optional[str] = None
):
    return {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Cookie": cookie_str or "",
        "DNT": "1",
        "Host": host_str,
        "Pragma": "no-cache",
        "Sec-CH-UA": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    }


def get_historical_treasury_auctions(
    xlsx_path: Optional[str] = None, return_df=False
) -> List[JSON] | pd.DataFrame:
    def get_treasury_query_sizing() -> List[str]:
        MAX_TREASURY_GOV_API_CONTENT_SIZE = 10000
        base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=1&page[size]=1"
        res = requests.get(base_url, headers=build_treasurydirect_header())
        if res.ok:
            meta = res.json()["meta"]
            size = meta["total-count"]
            number_requests = math.ceil(size / MAX_TREASURY_GOV_API_CONTENT_SIZE)
            return [
                f"https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]={i+1}&page[size]={MAX_TREASURY_GOV_API_CONTENT_SIZE}"
                for i in range(0, number_requests)
            ]

    links = get_treasury_query_sizing()

    async def fetch(session, url):
        async with session.get(url) as response:
            json_data = await response.json()
            return json_data["data"]

    async def run_fetch(urls):
        async with aiohttp.ClientSession(headers=build_treasurydirect_header()) as session:
            tasks = [fetch(session, url) for url in urls]
            return await asyncio.gather(*tasks)

    results: List[List[JSON]] = asyncio.run(run_fetch(links))
    flat = [item for sublist in results for item in sublist]
    if xlsx_path:
        df = pd.DataFrame(flat)
        df.to_excel(xlsx_path, index=False)
        if return_df:
            return df
    return flat

In [21]:
# n = 0 >> on-the-runs
def get_last_n_off_the_run_cusips(n=0, filtered=False, auctions_json: Optional[JSON] = None) -> List[Dict[str, str]]:
    auctions_json = auctions_json or get_historical_treasury_auctions()
    auctions_df = pd.DataFrame(auctions_json)
    auctions_df = auctions_df[
        (auctions_df["security_type"] != "TIPS")
        & (auctions_df["security_type"] != "TIPS Note")
        & (auctions_df["security_type"] != "TIPS Bond")
        & (auctions_df["security_type"] != "FRN")
        & (auctions_df["security_type"] != "FRN Note")
        & (auctions_df["security_type"] != "FRN Bond")
        & (auctions_df["security_type"] != "CMB")
    ]
    auctions_df = auctions_df.drop(
        auctions_df[
            (auctions_df["security_type"] == "Bill")
            & (
                auctions_df["original_security_term"]
                != auctions_df["security_term_week_year"]
            )
        ].index
    )
    auctions_df["auction_date"] = pd.to_datetime(auctions_df["auction_date"])
    current_date = pd.Timestamp.now()
    auctions_df = auctions_df[auctions_df["auction_date"] <= current_date]

    auctions_df["issue_date"] = pd.to_datetime(auctions_df["issue_date"])
    auctions_df = auctions_df.sort_values("issue_date", ascending=False)

    mapping = {
        "17-Week": 0.25,
        "26-Week": 0.5,
        "52-Week": 1,
        "2-Year": 2,
        "3-Year": 3,
        "5-Year": 5,
        "7-Year": 7,
        "10-Year": 10,
        "20-Year": 20,
        "30-Year": 30,
    }

    on_the_run = auctions_df.groupby("original_security_term").first().reset_index()
    on_the_run_result = on_the_run[
        [
            "original_security_term",
            "security_type",
            "cusip",
            "auction_date",
            "issue_date",
        ]
    ]

    off_the_run = auctions_df[~auctions_df.index.isin(on_the_run.index)]
    off_the_run_result = (
        off_the_run.groupby("original_security_term")
        .nth(list(range(1, n + 1)))
        .reset_index()
    )

    combined_result = pd.concat(
        [on_the_run_result, off_the_run_result], ignore_index=True
    )
    combined_result = combined_result.sort_values(
        by=["original_security_term", "issue_date"], ascending=[True, False]
    )

    combined_result["target_tenor"] = combined_result["original_security_term"].replace(
        mapping
    )
    mask = combined_result["original_security_term"].isin(mapping.keys())
    mapped_and_filtered_df = combined_result[mask]
    grouped = mapped_and_filtered_df.groupby("original_security_term")
    max_size = grouped.size().max()
    wrapper = []
    for i in range(max_size):
        sublist = []
        for _, group in grouped:
            if i < len(group):
                sublist.append(group.iloc[i].to_dict())
        sublist = sorted(sublist, key=lambda d: d["target_tenor"])
        if filtered:
            wrapper.append(
                {
                    auctioned_dict["target_tenor"]: auctioned_dict["cusip"]
                    for auctioned_dict in sublist
                }
            )
        else:
            wrapper.append(sublist)

    return wrapper


In [22]:
auctions_json = get_historical_treasury_auctions()

In [23]:
get_last_n_off_the_run_cusips(n=5, filtered=True, auctions_json=auctions_json)

C:\Users\chris\AppData\Local\Temp\ipykernel_28452\2866141031.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_result["target_tenor"] = combined_result["original_security_term"].replace(


[{0.25: '912797MN4',
  0.5: '912797MK0',
  1: '912797MG9',
  2: '91282CLB5',
  3: '91282CLG4',
  5: '91282CLC3',
  7: '91282CLD1',
  10: '91282CLF6',
  20: '912810UB2',
  30: '912810UC0'},
 {0.25: '912797ME4',
  0.5: '912797MJ3',
  1: '912797LW5',
  2: '91282CKY6',
  3: '91282CKZ3',
  5: '91282CKX8',
  7: '91282CKW0',
  10: '91282CKQ3',
  20: '912810UB2',
  30: '912810UA4'},
 {0.25: '912797MD6',
  0.5: '912797LZ8',
  1: '912797LN5',
  2: '91282CKS9',
  3: '91282CKV2',
  5: '91282CKT7',
  7: '91282CKU4',
  10: '91282CKQ3',
  20: '912810UB2',
  30: '912810UA4'},
 {0.25: '912797MC8',
  0.5: '912797LY1',
  1: '912797LB1',
  2: '91282CKK6',
  3: '91282CKR1',
  5: '91282CKP5',
  7: '91282CKN0',
  10: '91282CKQ3',
  20: '912810TZ1',
  30: '912810UA4'},
 {0.25: '912797MB0',
  0.5: '912797LX3',
  1: '912797KS5',
  2: '91282CKH3',
  3: '91282CKJ9',
  5: '91282CKG5',
  7: '91282CKF7',
  10: '91282CJZ5',
  20: '912810TZ1',
  30: '912810TX6'},
 {0.25: '912797MA2',
  0.5: '912797LR6',
  1: '912797KJ

In [25]:
from datetime import datetime
from pandas.tseries.offsets import BDay


t: pd.Timestamp = (datetime.today() + BDay(2))
t.to_pydatetime()

datetime.datetime(2024, 8, 14, 21, 13, 1, 476619)